Some notes while converting the comparison plots from the `cea plots-comparisons` to the dashboard.

In [37]:
import cea.config
import cea.inputlocator
reload(cea.config)
config = cea.config.Configuration()
locator = cea.inputlocator.InputLocator(scenario=config.scenario)

In [38]:
import pandas as pd
import os

from cea.analysis.multicriteria.optimization_post_processing.locating_individuals_in_generation_script import \
    get_pointers_to_correct_individual_generation

In [39]:
config.plots_scenario_comparisons.base_scenario

('baseline', None, None)

In [40]:
project = config.plots_scenario_comparisons.project
scenario_baseline = config.plots_scenario_comparisons.base_scenario

scenarios_names = [scenario_baseline] + config.plots_scenario_comparisons.scenarios
network_type = config.plots_scenario_comparisons.network_type

generation_base = scenario_baseline[1]
individual_base = scenario_baseline[2]

scenario_base_path = os.path.join(project, scenario_baseline[0])
scenarios_path = [os.path.join(project, scenario[0]) for scenario in
                  config.plots_scenario_comparisons.scenarios]


In [47]:
config.plots_scenario_comparisons.scenarios

[('baseline', '1', 'ind1'), ('baseline', '1', 'ind2')]

In [48]:
# scenarios = [[scenario_baseline]+ config.plots_scenario_comparisons.scenarios]
generations = [generation_base] + [scenario[1]
                                   for scenario in config.plots_scenario_comparisons.scenarios]
individuals = [individual_base] + [scenario[2]
                                   for scenario in config.plots_scenario_comparisons.scenarios]
categories = config.plots_scenario_comparisons.categories

generation_pointers, individual_pointers = pointers_all_scenarios(generations, individuals, [
    scenario_base_path] + scenarios_path)  # initialize class
category = "comparisons"

In [57]:
scenarios_path

['C:\\reference-case-cooling\\baseline',
 'C:\\reference-case-cooling\\baseline']

In [45]:
# adaptation of the function `cea.plots.comparisons.main.Plots#preprocessing_demand_scenarios`
def preprocessing_demand_scenarios(self):
    data_processed = pd.DataFrame()
    scenarios_clean = []
    for i, scenario_name in enumerate(self.scenarios_names):
        if scenario_name in scenarios_clean:
            scenario_name = scenario_name + "_duplicated_" + str(i)
        scenarios_clean.append(scenario_name)

    for scenario, scenario_name in zip(self.scenarios, scenarios_clean):
        locator = cea.inputlocator.InputLocator(scenario)
        data_raw = (pd.read_csv(locator.get_total_demand())[self.analysis_fields_demand + ["GFA_m2"]]).sum(axis=0)
        data_raw_df = pd.DataFrame({scenario_name: data_raw}, index=data_raw.index).T
        data_processed = data_processed.append(data_raw_df)
    return data_processed

def pointers_all_scenarios(generations, individuals, scenarios_path):
    generation_pointers = []
    individual_pointers = []
    for scenario_path, generation, individual in zip(scenarios_path, generations, individuals):
        if generation is None or individual is None:
            pointer_gen = "none"
            pointer_ind = "none"
        else:
            locator = cea.inputlocator.InputLocator(scenario_path)
            pointer_gen, pointer_ind = get_pointers_to_correct_individual_generation(generation,
                                                                                     individual, locator)
        generation_pointers.append(pointer_gen)
        individual_pointers.append(pointer_ind)
    return generation_pointers, individual_pointers

In [62]:
analysis_fields_demand = ["DH_hs_MWhyr", "DH_ww_MWhyr",
                                       'SOLAR_ww_MWhyr', 'SOLAR_hs_MWhyr',
                                       "DC_cs_MWhyr", 'DC_cdata_MWhyr', 'DC_cre_MWhyr',
                                       'PV_MWhyr', 'GRID_MWhyr',
                                       'NG_hs_MWhyr',
                                       'COAL_hs_MWhyr',
                                       'OIL_hs_MWhyr',
                                       'WOOD_hs_MWhyr',
                                       'NG_ww_MWhyr',
                                       'COAL_ww_MWhyr',
                                       'OIL_ww_MWhyr',
                                       'WOOD_ww_MWhyr',
                                       "E_sys_MWhyr",
                                       "Qhs_sys_MWhyr", "Qww_sys_MWhyr",
                                       "Qcs_sys_MWhyr", 'Qcdata_sys_MWhyr', 'Qcre_sys_MWhyr',
                                       "Eal_MWhyr",
                                       "Epro_MWhyr",
                                       "Edata_MWhyr",
                                       "E_cs_MWhyr",
                                       "E_hs_MWhyr",
                                       "E_ww_MWhyr",
                                       "Eaux_MWhyr",
                                       "E_cdata_MWhyr",
                                       "E_cre_MWhyr"]

In [65]:
[scenario_base_path] + scenarios_path

['C:\\reference-case-cooling\\baseline',
 'C:\\reference-case-cooling\\baseline',
 'C:\\reference-case-cooling\\baseline']

In [68]:
data_processed = pd.DataFrame()
for i, scenario in enumerate([scenario_base_path] + scenarios_path):
    locator = cea.inputlocator.InputLocator(scenario)
    data_raw = (pd.read_csv(locator.get_total_demand())[analysis_fields_demand + ["GFA_m2"]]).sum(axis=0)
    data_raw_df = pd.DataFrame({str(i): data_raw}, index=data_raw.index).T
    data_processed = data_processed.append(data_raw_df)
data_processed

,DH_hs_MWhyr,DH_ww_MWhyr,SOLAR_ww_MWhyr,SOLAR_hs_MWhyr,DC_cs_MWhyr,DC_cdata_MWhyr,DC_cre_MWhyr,PV_MWhyr,GRID_MWhyr,NG_hs_MWhyr,...,Eal_MWhyr,Epro_MWhyr,Edata_MWhyr,E_cs_MWhyr,E_hs_MWhyr,E_ww_MWhyr,Eaux_MWhyr,E_cdata_MWhyr,E_cre_MWhyr,GFA_m2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96609.577,0.0,...,46561.64,0.0,0.0,48272.801,0.0,1455.602,319.535,0.0,0.0,582543.344
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96609.577,0.0,...,46561.64,0.0,0.0,48272.801,0.0,1455.602,319.535,0.0,0.0,582543.344
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96609.577,0.0,...,46561.64,0.0,0.0,48272.801,0.0,1455.602,319.535,0.0,0.0,582543.344


In [74]:
def sum_total_demand(scenario):
    locator = cea.inputlocator.InputLocator(scenario)
    return (pd.read_csv(locator.get_total_demand())[analysis_fields_demand + ["GFA_m2"]]).sum(axis=0)
pd.DataFrame({i: sum_total_demand(scenario) for i, scenario in enumerate([scenario_base_path] + scenarios_path)})

,0,1,2
DH_hs_MWhyr,0.000,0.000,0.000
DH_ww_MWhyr,0.000,0.000,0.000
SOLAR_ww_MWhyr,0.000,0.000,0.000
SOLAR_hs_MWhyr,0.000,0.000,0.000
DC_cs_MWhyr,0.000,0.000,0.000
DC_cdata_MWhyr,0.000,0.000,0.000
DC_cre_MWhyr,0.000,0.000,0.000
PV_MWhyr,0.000,0.000,0.000
GRID_MWhyr,96609.577,96609.577,96609.577
NG_hs_MWhyr,0.000,0.000,0.000


In [73]:
(pd.read_csv(locator.get_total_demand())[analysis_fields_demand + ["GFA_m2"]]).sum(axis=0)

DH_hs_MWhyr              0.000
DH_ww_MWhyr              0.000
SOLAR_ww_MWhyr           0.000
SOLAR_hs_MWhyr           0.000
DC_cs_MWhyr              0.000
DC_cdata_MWhyr           0.000
DC_cre_MWhyr             0.000
PV_MWhyr                 0.000
GRID_MWhyr           96609.577
NG_hs_MWhyr              0.000
COAL_hs_MWhyr            0.000
OIL_hs_MWhyr             0.000
WOOD_hs_MWhyr            0.000
NG_ww_MWhyr              0.000
COAL_ww_MWhyr            0.000
OIL_ww_MWhyr             0.000
WOOD_ww_MWhyr            0.000
E_sys_MWhyr          46881.177
Qhs_sys_MWhyr            0.000
Qww_sys_MWhyr         1310.041
Qcs_sys_MWhyr       215911.464
Qcdata_sys_MWhyr         0.000
Qcre_sys_MWhyr           0.000
Eal_MWhyr            46561.640
Epro_MWhyr               0.000
Edata_MWhyr              0.000
E_cs_MWhyr           48272.801
E_hs_MWhyr               0.000
E_ww_MWhyr            1455.602
Eaux_MWhyr             319.535
E_cdata_MWhyr            0.000
E_cre_MWhyr              0.000
GFA_m2  

In [9]:
config.plots_scenario_comparisons.base_scenario

'C:\\reference-case-cooling\\baseline'